In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
%cd /content/drive/My Drive/Colab Notebooks/Github/predicting_components_source/PredictingComponent-multilabel
!ls

/content/drive/My Drive/Colab Notebooks/Github/predicting_components_source/PredictingComponent-multilabel
Classifier  experiment_predict_component_result.csv  NCE       test2.txt
Dataset     __init__.py				     Other_DL


**Set experiment parameters**

In [0]:
project = 'CONF'

trainingSize = 60
validationSize = 20
testSize = 20

**Load csv of the whole dataset**

In [0]:
import numpy as np
import pandas as pd

# /content/drive/My Drive/Colab Notebooks/Github/predicting_components_source/PredictingComponent-multilabel/Dataset/Data/HBASE.csv

data_path = "Dataset/Data/" + project + ".csv"
data_path
data = pd.read_csv(data_path, encoding= 'unicode_escape')


In [39]:
nan_values = data.isna()
nan_columns = nan_values.any()

columns_with_nan = data.columns[nan_columns].tolist()
print(columns_with_nan)
data = data.fillna("None")
nan_values = data.isna()
nan_columns = nan_values.any()

columns_with_nan = data.columns[nan_columns].tolist()
print(columns_with_nan)

[]
[]


In [0]:
# data.head(3)

**Remove unused columns and concat summary and description**

In [0]:
data['description'] = data['title'] + data['description']
data = data.drop('title',1) 
data = data.drop('issuekey',1) 
data = data.drop('component',1) 

In [42]:
data.head(3)

,description,Add-ons - Confluence Questions,Add-ons - Team Calendar,Administration,Administration - Application Links,Administration - Look and Feel / Themes,Administration - Universal Plugin Manager,Analytics,Backup / Restore,Business Teams,Clustering,Content - Attachments,Content - Blogs,Content - Blueprints / Templates,Content - Favourites,Content - History,Content - Inline Comments,Content - Labels,Content - Likes,Content - Linking,Content - Page,Content - Page Comments,Content - Restrictions,Content - Sharing,Content - Trackback,Core - APIs,Core - Analytics,Core - Email,Core - Macros Platform,Core - OSGi Plugins Platform,Core - Web Resources,Core - Workbox,Dashboard,Database / Hibernate,Documentation,Documentation - All,Editor,Editor - Attachment,Editor - Collaborative editing,Editor - Drafts,...,Refactorings,Search - Core,Search - Indexing,Searching / Indexing,Security,Server - Administration,Server - Crowd,Server - Data Center,Server - Installer / Setup,Server - Mail Archiving,Server - Platform,Server - Plugin ï¿½Development,Server - Source Distribution,Server - Support Tools,Server - Upgrade Manager,Setup / Installation,Site - Backup / Import,Space - Blueprints,Space - Export / Import,Space - Personal Space,Space Sidebar,Spaces,Themes - Documentation,Upgrade Manager,User - Follow,User - Global / Space Permissions,User - Groups,User - Management,User - Mentions,User - Profile,User Interface,Users & Groups,Web Interface,Web Interface - AUI,Web Interface - Keyboard Navigation,WebDAV - Backups,WebDAV - Editing,Workbox,zzOLD Browser Support,zzOLD Business Teams
0,Customisable dashboard with portletsWe would l...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Upgrading from A1 to A2 throws exception...Aft...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Show descriptions of search result matchesWe s...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
data.description[0]

"Customisable dashboard with portletsWe would like to have :    * Reusable sub page components (portlets) some provided by atlassian, some written by us ( We can't wait for you to do all the work , and neither can you :-)    * All portlets should be user editable - made of html + velocity a la roller    * Flexible Interactive Page Layout - Similar to the JIRA dashboard  some pre defined e.g Dashboard, Wiki, Space Home,...    * Customisable Navigation - Both meta data driven , and static using a hand written nav portlet    * Multiple built-in sub-page(portlet) types ( RSS Agg, Calendar , Free Form ) ( as mentioned above they should still be user editable, and be htnl+velocity based )    Attached is a mockup of the portlet based dashboard ."

**Split training/test set**

In [44]:
if trainingSize + validationSize + testSize == 100:
  numData = len(data)
  numTrain = int((trainingSize * numData) / 100)
  numValidation = int((validationSize * numData) / 100)
  numTest = int((testSize * numData) / 100)

  print("#Total size: %s" % numData)
  print("#Training : %s, #Validation : %s, #Testing : %s" % (numTrain, numValidation, numTest))
  print("Total: %s" % (numTrain + numValidation + numTest))

  firstStop = numTrain
  secondStop = numTrain + numValidation
else:
  print("input not valid")


#Total size: 11000
#Training : 6600, #Validation : 2200, #Testing : 2200
Total: 11000


In [45]:
divided_set = np.zeros([numData, 3]).astype(int)
divided_set[0:firstStop, 0] = 1
divided_set[firstStop:secondStop, 1] = 1
divided_set[secondStop:numData, 2] = 1
experimentalSet = pd.DataFrame(divided_set, columns = ['train', 'validate', 'test'])
print(experimentalSet)

       train  validate  test
0          1         0     0
1          1         0     0
2          1         0     0
3          1         0     0
4          1         0     0
...      ...       ...   ...
10995      0         0     1
10996      0         0     1
10997      0         0     1
10998      0         0     1
10999      0         0     1

[11000 rows x 3 columns]


In [46]:
trainSetIndex = experimentalSet['train'].value_counts()[1]
validSetIndex = experimentalSet['validate'].value_counts()[1]
testSetIndex = experimentalSet['test'].value_counts()[1]
print("Training set index: " + str(trainSetIndex))
print("Validation set index: " + str(validSetIndex))
print("Test set index: " + str(testSetIndex))

Training set index: 6600
Validation set index: 2200
Test set index: 2200


In [47]:
trainSetData = data.iloc[:trainSetIndex, :]
validSetData = data.iloc[trainSetIndex:trainSetIndex + validSetIndex, :]
testSetData = data.iloc[trainSetIndex + validSetIndex:, :]

print("Training set:")
trainSetData.head(3)


Training set:


,description,Add-ons - Confluence Questions,Add-ons - Team Calendar,Administration,Administration - Application Links,Administration - Look and Feel / Themes,Administration - Universal Plugin Manager,Analytics,Backup / Restore,Business Teams,Clustering,Content - Attachments,Content - Blogs,Content - Blueprints / Templates,Content - Favourites,Content - History,Content - Inline Comments,Content - Labels,Content - Likes,Content - Linking,Content - Page,Content - Page Comments,Content - Restrictions,Content - Sharing,Content - Trackback,Core - APIs,Core - Analytics,Core - Email,Core - Macros Platform,Core - OSGi Plugins Platform,Core - Web Resources,Core - Workbox,Dashboard,Database / Hibernate,Documentation,Documentation - All,Editor,Editor - Attachment,Editor - Collaborative editing,Editor - Drafts,...,Refactorings,Search - Core,Search - Indexing,Searching / Indexing,Security,Server - Administration,Server - Crowd,Server - Data Center,Server - Installer / Setup,Server - Mail Archiving,Server - Platform,Server - Plugin ï¿½Development,Server - Source Distribution,Server - Support Tools,Server - Upgrade Manager,Setup / Installation,Site - Backup / Import,Space - Blueprints,Space - Export / Import,Space - Personal Space,Space Sidebar,Spaces,Themes - Documentation,Upgrade Manager,User - Follow,User - Global / Space Permissions,User - Groups,User - Management,User - Mentions,User - Profile,User Interface,Users & Groups,Web Interface,Web Interface - AUI,Web Interface - Keyboard Navigation,WebDAV - Backups,WebDAV - Editing,Workbox,zzOLD Browser Support,zzOLD Business Teams
0,Customisable dashboard with portletsWe would l...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Upgrading from A1 to A2 throws exception...Aft...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Show descriptions of search result matchesWe s...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
print("Validation set:")
validSetData.head(3)

Validation set:


,description,Add-ons - Confluence Questions,Add-ons - Team Calendar,Administration,Administration - Application Links,Administration - Look and Feel / Themes,Administration - Universal Plugin Manager,Analytics,Backup / Restore,Business Teams,Clustering,Content - Attachments,Content - Blogs,Content - Blueprints / Templates,Content - Favourites,Content - History,Content - Inline Comments,Content - Labels,Content - Likes,Content - Linking,Content - Page,Content - Page Comments,Content - Restrictions,Content - Sharing,Content - Trackback,Core - APIs,Core - Analytics,Core - Email,Core - Macros Platform,Core - OSGi Plugins Platform,Core - Web Resources,Core - Workbox,Dashboard,Database / Hibernate,Documentation,Documentation - All,Editor,Editor - Attachment,Editor - Collaborative editing,Editor - Drafts,...,Refactorings,Search - Core,Search - Indexing,Searching / Indexing,Security,Server - Administration,Server - Crowd,Server - Data Center,Server - Installer / Setup,Server - Mail Archiving,Server - Platform,Server - Plugin ï¿½Development,Server - Source Distribution,Server - Support Tools,Server - Upgrade Manager,Setup / Installation,Site - Backup / Import,Space - Blueprints,Space - Export / Import,Space - Personal Space,Space Sidebar,Spaces,Themes - Documentation,Upgrade Manager,User - Follow,User - Global / Space Permissions,User - Groups,User - Management,User - Mentions,User - Profile,User Interface,Users & Groups,Web Interface,Web Interface - AUI,Web Interface - Keyboard Navigation,WebDAV - Backups,WebDAV - Editing,Workbox,zzOLD Browser Support,zzOLD Business Teams
6600,typing {quote} to create a quote block does no...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6601,Wrong Information Regarding Export Space to PD...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6602,Confluence Creates Duplicate Groups Internally...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
print("testset:")
testSetData.head(3)

testset:


,description,Add-ons - Confluence Questions,Add-ons - Team Calendar,Administration,Administration - Application Links,Administration - Look and Feel / Themes,Administration - Universal Plugin Manager,Analytics,Backup / Restore,Business Teams,Clustering,Content - Attachments,Content - Blogs,Content - Blueprints / Templates,Content - Favourites,Content - History,Content - Inline Comments,Content - Labels,Content - Likes,Content - Linking,Content - Page,Content - Page Comments,Content - Restrictions,Content - Sharing,Content - Trackback,Core - APIs,Core - Analytics,Core - Email,Core - Macros Platform,Core - OSGi Plugins Platform,Core - Web Resources,Core - Workbox,Dashboard,Database / Hibernate,Documentation,Documentation - All,Editor,Editor - Attachment,Editor - Collaborative editing,Editor - Drafts,...,Refactorings,Search - Core,Search - Indexing,Searching / Indexing,Security,Server - Administration,Server - Crowd,Server - Data Center,Server - Installer / Setup,Server - Mail Archiving,Server - Platform,Server - Plugin ï¿½Development,Server - Source Distribution,Server - Support Tools,Server - Upgrade Manager,Setup / Installation,Site - Backup / Import,Space - Blueprints,Space - Export / Import,Space - Personal Space,Space Sidebar,Spaces,Themes - Documentation,Upgrade Manager,User - Follow,User - Global / Space Permissions,User - Groups,User - Management,User - Mentions,User - Profile,User Interface,Users & Groups,Web Interface,Web Interface - AUI,Web Interface - Keyboard Navigation,WebDAV - Backups,WebDAV - Editing,Workbox,zzOLD Browser Support,zzOLD Business Teams
8800,Export of page which includes other pages cont...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8801,Creating 2 spaces with same ID doesn't produce...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8802,5.7 Comments Not Working on Headings when othe...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**Wording embedding layer using pretrained model from Google's word2vec**

In [50]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
trainText = trainSetData.description
validText = validSetData.description
testText = testSetData.description

In [52]:
NUM_WORDS=300000
tokenizer = Tokenizer(num_words=NUM_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'', lower=True)
tokenizer.fit_on_texts(trainText)
trainSequences = tokenizer.texts_to_sequences(trainText)
validSequences = tokenizer.texts_to_sequences(validText)
testSequences = tokenizer.texts_to_sequences(testText)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 29873 unique tokens.


In [53]:
x_train = pad_sequences(trainSequences)
y_train = trainSetData.iloc[:,1:].values

x_valid = pad_sequences(validSequences, maxlen = x_train.shape[1])
y_valid = validSetData.iloc[:,1:].values

x_test = pad_sequences(testSequences,maxlen = x_train.shape[1])
y_test = testSetData.iloc[:,1:].values

print('Shape of X train and X validation tensor:', x_train.shape, x_valid.shape)
print('Shape of label train and validation tensor:', y_train.shape, y_valid.shape)


print('Shape of X test:', x_test.shape)
print('Shape of label test tensor:', y_test.shape)

Shape of X train and X validation tensor: (6600, 3300) (2200, 3300)
Shape of label train and validation tensor: (6600, 128) (2200, 128)
Shape of X test: (2200, 3300)
Shape of label test tensor: (2200, 128)


**word embedding**

In [0]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

In [55]:
word_vectors = KeyedVectors.load_word2vec_format('Other_DL/pretrainedw2v/GoogleNews-vectors-negative300.bin.gz', 
                                                 binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [56]:
EMBEDDING_DIM=300
vocabulary_size = min(len(word_index)+1,NUM_WORDS) # minimum fo word index and num words
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

count_word_not_found = 0
for word, i in word_index.items():
  if i>=NUM_WORDS:
    continue
  try:
    embedding_vector = word_vectors[word]
    embedding_matrix[i] = embedding_vector
  except KeyError:
    embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)
    count_word_not_found = count_word_not_found + 1

print("word not found in pretrained w2v: " + str(count_word_not_found))
# there are many words do not exist in google w2v. we random number for those words.

del(word_vectors)


word not found in pretrained w2v: 19679


**Construct CNN model**

In [57]:
from keras.layers import Embedding

embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)
                            
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau


sequence_length = x_train.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=len(y_train[0]), activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3300)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 3300, 300)    8962200     input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 3300, 300, 1) 0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 3298, 1, 100) 90100       reshape_1[0][0]                  
____________________________________________________________________________________________

**Train the model**

In [58]:
path_modelcheckpoint = 'Other_DL/model_checkpoint/'
adam = Adam(lr=1e-3)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
model_checkpoint = ModelCheckpoint(path_modelcheckpoint+project+'_mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')


model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])

training_history = model.fit(x_train, y_train, batch_size=100, epochs=100, verbose=1, validation_data=(x_valid, y_valid), 
                             callbacks=[early_stopping, model_checkpoint, reduce_lr_loss])  # starts training


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6600 samples, validate on 2200 samples
Epoch 1/100
6600/6600 [==============================] - 1453s 220ms/step - loss: 4.3683 - acc: 0.3124 - val_loss: 3.6907 - val_acc: 0.4759

Epoch 00001: val_loss improved from inf to 3.69074, saving model to Other_DL/model_checkpoint/CONF_mdl_wts.hdf5
Epoch 2/100
6600/6600 [==============================] - 1452s 220ms/step - loss: 2.9503 - acc: 0.5021 - val_loss: 3.2428 - val_acc: 0.5045

Epoch 00002: val_loss improved from 3.69074 to 3.24278, saving model to Other_DL/model_checkpoint/CONF_mdl_wts.hdf5
Epoch 3/100
6600/6600 [==============================] - 1444s 219ms/step - loss: 2.5106 - acc: 0.5877 - val_loss: 3.1133 - val_acc: 0.5159

Epoch 00003: val_loss improved from 3.24278 to 3.11328, saving model to Other_DL/model_checkpoint/CONF_mdl_wts.hdf5
Epoch 4/100
6600/6600 [==============================] - 1453s 220ms/step - loss: 2.2734 - acc: 0.6458 - val_loss: 3.0412 - val_acc: 0.5368

Epoch 00004: val_loss improved from 3.11328 

**Save model**

In [59]:
import pandas as pd
path_history = 'Other_DL/training_history/'
path_model_json = 'Other_DL/model_structure/'
path_model_weight = 'Other_DL/model_weight/'

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(training_history.history) 

# or save to csv: 
hist_csv_file = path_history+project+'_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# save the best model
# save model structure
model_json = model.to_json()
with open(path_model_json+project+"_model.json", "w") as json_file:
    json_file.write(model_json)
# save model weight
model.save_weights(path_model_weight+project+"_wieght.h5")
print("Saved model to disk")


Saved model to disk


In [60]:
# # later...

from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy as np
import pandas as pd

path_history = 'Other_DL/training_history/'
path_model_json = 'Other_DL/model_structure/'
path_model_weight = 'Other_DL/model_weight/'

# # load json and create model
json_file = open(path_model_json+project+"_model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# # load weights into new model
model.load_weights(path_model_weight+project+"_wieght.h5")
print("Loaded model from disk")

# # evaluate loaded model on test data
# loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk


**Make a recommendation on the testset**

In [0]:
y_pred = model.predict(x_test)

***Save result***

In [0]:
path_log = 'Other_DL/output/log/'

np.savetxt(path_log + project + "_actual.csv", y_test, delimiter=",")
np.savetxt(path_log + project + "_estimate.csv", y_pred, delimiter=",")

***Evaluation***

In [0]:
def recall(actual, estimate, startK, stopK, stepK):
    recall_k = []
    for k in range(startK, stopK + 1, stepK):
        if k == 0:
            k = 1
        m = len(actual)
        sum_recall = 0.0
        for j in range(len(actual)):
            y_true = np.argwhere(actual[j])
            y_pred = estimate[j].argsort()[-k:]
            # print y_true
            # print y_pred
            intersect = (y_true == y_pred).sum()
            # print intersect
            # print (len(y_true))
            # print intersect / float(len(y_true))
            sum_recall = sum_recall + (intersect / float(len(y_true)))
            # print sum_recall
        recall_k.append(sum_recall / float(m))
        # print m
        print('Recall@' + str(k) + ': {:.4f}'.format(sum_recall / float(m)))
    return recall_k

In [64]:
startK = 1
stepK = 1
stopK = 30

print("Project:" + project)

path_output = 'Other_DL/output/'

recall_k = recall(y_test, y_pred, startK, stopK, stepK)

np.savetxt(path_output + project + "_recall_" + str(startK) + "_" + str(stopK)+ "_v.csv", recall_k, delimiter=",", fmt='%1.4f')
with open(path_output + "performance" + "_recall_" + str(startK) + "_" + str(stopK)+ ".csv", 'a') as myoutput:
  myoutput.write(project + "," + ",".join(map(str, recall_k)) + '\n')



Project:CONF
Recall@1: 0.4656
Recall@2: 0.5624
Recall@3: 0.5979
Recall@4: 0.6147
Recall@5: 0.6233
Recall@6: 0.6331
Recall@7: 0.6412
Recall@8: 0.6478
Recall@9: 0.6505
Recall@10: 0.6555
Recall@11: 0.6612
Recall@12: 0.6683
Recall@13: 0.6710
Recall@14: 0.6737
Recall@15: 0.6764
Recall@16: 0.6796
Recall@17: 0.6823
Recall@18: 0.6846
Recall@19: 0.6901
Recall@20: 0.6928
Recall@21: 0.6937
Recall@22: 0.6983
Recall@23: 0.7028
Recall@24: 0.7080
Recall@25: 0.7110
Recall@26: 0.7183
Recall@27: 0.7233
Recall@28: 0.7278
Recall@29: 0.7333
Recall@30: 0.7378
